In [1]:
import os
import pandas as pd
import numpy as np
from datetime import datetime
from scipy import stats
import matplotlib
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns

%matplotlib inline
cwd = os.getcwd()
print("Current working directory: {0}".format(cwd))

HousePrice1 = pd.read_csv("HousePrice1.csv")

Current working directory: /Users/chereowang27/NYCDSA/Machine_Learning_I/Machine_Learning_Project/Ames_Housing_ML


In [2]:
HousePrice1['ppsqft']= HousePrice1["SalePrice"]/HousePrice1['GrLivArea']
HousePrice1['mo_Yr_sold'] = HousePrice1.MoSold.astype(str) + '/' + HousePrice1.YrSold.astype(str)
HousePrice1['mo_Yr_sold'] = pd.to_datetime(HousePrice1['mo_Yr_sold'])
HousePrice1_gr = HousePrice1.groupby(['mo_Yr_sold','Neighborhood'])[['SalePrice']].median()\
         .reset_index()
HousePrice1_gr

,mo_Yr_sold,Neighborhood,SalePrice
0,2006-01-01,CollgCr,232500.0
1,2006-01-01,Gilbert,172400.0
2,2006-01-01,NAmes,138000.0
3,2006-01-01,NWAmes,181000.0
4,2006-01-01,SWISU,80000.0
...,...,...,...
891,2010-07-01,Crawfor,257500.0
892,2010-07-01,Edwards,80000.0
893,2010-07-01,IDOTRR,55000.0
894,2010-07-01,NAmes,161000.0


In [3]:
HousePrice2 = HousePrice1.drop(['Unnamed: 0', 'MapRefNo','PID','Unnamed: 0.1', 'Prop_Addr'], \
                               axis = 1)
num_features = HousePrice2._get_numeric_data()  \
 .drop(['MSSubClass', 'SalePrice'], axis = 1)
num_features['bath'] = num_features['FullBath'] + 0.5*num_features['HalfBath']
numCol = num_features.columns

In [4]:
col = HousePrice2.columns
catCol = [col for col in HousePrice2.columns if col not in num_features.columns]

In [5]:
cat_features = HousePrice1[catCol]
# group Neighborhood
p_mean_by_nb = HousePrice1.groupby('Neighborhood')['SalePrice'].mean() 
nb_bin = pd.qcut(p_mean_by_nb, q = [0,0.25,.5,.75,1],
                 labels = ['tier1','tier2','tier3','tier4'])\
                .to_dict()
                 #tier1 < tier2 < tier3 < tier4
cat_features['NbPriceTier'] = cat_features['Neighborhood'].map(nb_bin)       

<ipython-input-5-4102d6c37cf1>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cat_features['NbPriceTier'] = cat_features['Neighborhood'].map(nb_bin)


In [6]:
features = pd.concat([num_features, cat_features], axis = 1) \
           .drop(['GrLivArea','FullBath', 'HalfBath', \
                  'BsmtFullBath', 'BsmtHalfBath', \
                  'BsmtFinSF1','SalePrice','LandSlope','ExterQual', \
                  'Condition1','Condition2', \
                  'BsmtFinType1','BsmtFinType2', \
                  'BsmtQual', 'GarageQual','MoSold','mo_Yr_sold',   \
                  'BsmtFinSF2', 'BsmtUnfSF','Neighborhood','ppsqft'],axis = 1)

In [9]:
catCol = [col for col in features.columns if col not in num_features.columns]
features= pd.get_dummies(features, columns = catCol, drop_first = True)

y = HousePrice1['SalePrice']
HousePrice1['log_Price'] = np.log10(HousePrice1['SalePrice'])
logy = HousePrice1['log_Price']

In [22]:
from sklearn import linear_model, datasets
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import time

ridge = Ridge()
lasso = Lasso()
# net = ElasticNet()
from sklearn import datasets
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
# X_train, X_test, y_train, y_test = train_test_split(features, y, test_size = 0.3, random_state = 0)

X_train_log, X_test_log, y_train_log, y_test_log = train_test_split(features, logy, test_size = 0.3, random_state = 0)

In [29]:
lasso_pipe = Pipeline([
    ('standardize', StandardScaler()),
    ('lasso',lasso)
])
params = {
#     'lasso__alpha': [0.0001, 0.001, 0.1, 1, 10]     .001
#     'lasso__alpha': np.logspace(-4,-3,100)          .001
#    'lasso_max_iter': [5000],
     }
kfold = KFold(n_splits = 3, shuffle = True)
gs_log = GridSearchCV(lasso_pipe, param_grid = params, scoring = 'r2', cv = kfold)

start = time.time()
gs_log.fit(X_train_log, y_train_log)
print("Estimator which gave the highest score: ", gs_log.best_estimator_)
print("Mean cross-validated score of the best_estimator: ", gs_log.best_score_)
print("The best parameters are: ", gs_log.best_params_)

Estimator which gave the highest score:  Pipeline(steps=[('standardize', StandardScaler()),
                ('lasso', Lasso(alpha=0.001))])
Mean cross-validated score of the best_estimator:  0.8887602702925755
The best parameters are:  {'lasso__alpha': 0.001}


In [30]:
# Run Lasso Regression with the best alpha from Grid search
lasso.set_params(alpha = 0.1)
alphas = []
lasso.set_params(normalize = False)
coefs_lasso = []

#for alpha in alphas:
lasso.set_params(alpha = np.log(0.001))
lasso.fit(X_train_log, y_train_log)
lasso_coefs = dict(zip(features.columns, lasso.coef_))
lasso_coefs

lasso_coefs_keep = {k:v for (k,v) in lasso_coefs.items() if v !=0}
lasso_coefs_keep
 

{'LotFrontage': 24.80301620287424,
 'LotArea': -0.06821803261247858,
 'OverallQual': 311.0824843443822,
 'OverallCond': 143.74127026966667,
 'YearBuilt': 6.109950782908106,
 'YearRemodAdd': 16.825057613940352,
 'MasVnrArea': -1.12178367525159,
 'TotalBsmtSF': -0.08298713985534995,
 '1stFlrSF': -2.1876443372531944,
 '2ndFlrSF': -0.48446767805619445,
 'LowQualFinSF': 3.2231395829206297,
 'BedroomAbvGr': 82.34048835291377,
 'KitchenAbvGr': -1137.2513758680586,
 'TotRmsAbvGrd': -272.34557095532244,
 'Fireplaces': -352.0983012307111,
 'GarageYrBlt': -47.160024235056284,
 'GarageCars': 1224.3638712817942,
 'GarageArea': -0.4030867656844845,
 'WoodDeckSF': 1.256466514770462,
 'OpenPorchSF': -0.6420880257701095,
 'EnclosedPorch': -2.6458659567606673,
 '3SsnPorch': 2.968306929562571,
 'ScreenPorch': 0.12450548179830771,
 'PoolArea': -0.41928786463891266,
 'MiscVal': -0.4322326634711494,
 'YrSold': -139.8217709347176,
 'bath': 956.0590533251695,
 'MSSubClass_30': -2657.002460426405,
 'MSSubClass